In [18]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
%autoreload

import h2o
import create_data as create
import pandas as pd
import utilities as utils

from h2o.automl import H2OAutoML

In [20]:
data = create.CreateTrainingData()
train = data.create_training_data()
test = create.create_test_data()
target = create.create_target_data()

In [15]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,14 mins 20 secs
H2O_cluster_timezone:,Europe/Oslo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.4
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_stinky_6hnobf
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.889 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [21]:
X_train = h2o.H2OFrame(train)
y = 'pv_measurement'

X = X_train.columns
X.remove(y)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
autoML = H2OAutoML(max_models=30, seed=1, nfolds=5)
autoML.train(x=X, y=y, training_frame=train)

In [ ]:
# Remove features that are not in the training set
test_pd = test[X]

# Set nans to 0 in test_pd
# test_pd.fillna(0, inplace=True)

test = h2o.H2OFrame(test_pd)

prediction = autoML.predict(test)

In [ ]:
preds_df = prediction.as_data_frame()
preds_df[preds_df < 0] = 0

results = pd.concat([test_pd.reset_index(), preds_df], axis=1)
results['date_forecast'] = pd.to_datetime(results['date_forecast'])
results.set_index('date_forecast', inplace=True)
results.rename(columns={'predict': 'pv_prediction'}, inplace=True)

utils.save_to_csv(results, 'h2o_nans_to_0')